# Youtube Ranking - Only Selenium

In [1]:
!apt-get update > /dev/null
!pip install selenium > /dev/null
!apt install chromium-chromedriver > /dev/null

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [3]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument("--single-process")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome('chromedriver', options=options)

In [6]:
trs = driver.find_elements_by_css_selector('.aos-init')

In [7]:
def convert_unit(s):
    s = s.replace('억', '').replace('개', '').replace(',', '')
    s = s.replace('만', '0000')
    return f'{int(s):,d}'

In [10]:
from tqdm.notebook import tqdm

In [11]:
results = []
for page in tqdm(range(1, 11)):
    url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=' + str(page)
    driver.get(url)
    time.sleep(4)
    window_len = len(driver.window_handles)
    if window_len == 2:
        driver.switch_to.window(driver.window_handles[1])
        driver.close()
    driver.switch_to.window(driver.window_handles[0])
    body = driver.find_element_by_css_selector('body')
    for i in range(30):
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)
    trs = driver.find_elements_by_css_selector('.aos-init')
    for tr in trs:
        category = tr.find_element_by_css_selector('p.category').text.strip(' \n[]')
        name = tr.find_element_by_css_selector('.subject a').text.strip()
        subscriber = convert_unit(tr.find_element_by_css_selector('.subscriber_cnt').text)
        view = convert_unit(tr.find_element_by_css_selector('.view_cnt').text)
        video = convert_unit(tr.find_element_by_css_selector('.video_cnt').text)
        results.append([category, name, subscriber, view, video])

  0%|          | 0/10 [00:00<?, ?it/s]

In [12]:
df = pd.DataFrame(results, columns=['카테고리', '채널명', '구독자수', '조회수', '비디오'])
df.head()

,카테고리,채널명,구독자수,조회수,비디오
0,음악/댄스/가수,BLACKPINK,"64,000,000","19,003,810,000",371
1,음악/댄스/가수,HYBE LABELS,"60,300,000","18,713,050,000",654
2,음악/댄스/가수,BANGTANTV,"56,400,000","12,194,960,000","1,579"
3,음악/댄스/가수,SMTOWN,"28,500,000","21,870,920,000","3,729"
4,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],"26,500,000","11,052,880,000",223


In [13]:
df.tail()

,카테고리,채널명,구독자수,조회수,비디오
995,게임,미소,"510,000","210,770,000","3,084"
996,미분류,밉지않은 관종언니,"510,000","67,740,000",170
997,미분류,복지마블TV [Welfare Marble],"510,000","49,960,000",240
998,음식/요리/레시피,Muggo,"510,000","183,720,000","1,388"
999,게임,임선비,"510,000","234,290,000","1,134"
